Mixtures of Logit Swissmetro


Options for the few_draws.toml:
- Second derivative set to 0
- Number of draws: 100 (for now)

In [28]:
import pandas as pd
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, MonteCarlo, log, bioDraws, Variable
from biogeme.tools import TemporaryFile

Preparing the data

In [29]:
url = "http://transp-or.epfl.ch/data/swissmetro.dat"

# Read the data into a DataFrame
df = pd.read_csv(url, sep='\t')
database_swissmetro = db.Database('swissmetro', df)

#Definition of the variables:

GROUP = Variable('GROUP')
SURVEY = Variable('SURVEY')
SP = Variable('SP')
ID = Variable('ID')
PURPOSE = Variable('PURPOSE')
FIRST = Variable('FIRST')
TICKET = Variable('TICKET')
WHO = Variable('WHO')
LUGGAGE = Variable('LUGGAGE')
AGE = Variable('AGE')
MALE = Variable('MALE')
INCOME = Variable('INCOME')
GA = Variable('GA')
ORIGIN = Variable('ORIGIN')
DEST = Variable('DEST')
TRAIN_AV = Variable('TRAIN_AV')
CAR_AV = Variable('CAR_AV')
SM_AV = Variable('SM_AV')
TRAIN_TT = Variable('TRAIN_TT')
TRAIN_CO = Variable('TRAIN_CO')
TRAIN_HE = Variable('TRAIN_HE')
SM_TT = Variable('SM_TT')
SM_CO = Variable('SM_CO')
SM_HE = Variable('SM_HE')
SM_SEATS = Variable('SM_SEATS')
CAR_TT = Variable('CAR_TT')
CAR_CO = Variable('CAR_CO')
CHOICE = Variable('CHOICE')

#Removing observation ("Biogeme way")
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database_swissmetro.remove(exclude)

#Definition of new variables:
SM_COST = database_swissmetro.DefineVariable('SM_COST', SM_CO * (GA == 0))
TRAIN_COST = database_swissmetro.DefineVariable('TRAIN_COST', TRAIN_CO * (GA == 0))
CAR_AV_SP = database_swissmetro.DefineVariable('CAR_AV_SP', CAR_AV * (SP != 0))
TRAIN_AV_SP = database_swissmetro.DefineVariable('TRAIN_AV_SP', TRAIN_AV * (SP != 0))
TRAIN_TT_SCALED = database_swissmetro.DefineVariable('TRAIN_TT_SCALED', TRAIN_TT / 100)
TRAIN_COST_SCALED = database_swissmetro.DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100)
SM_TT_SCALED = database_swissmetro.DefineVariable('SM_TT_SCALED', SM_TT / 100)
SM_COST_SCALED = database_swissmetro.DefineVariable('SM_COST_SCALED', SM_COST / 100)
CAR_TT_SCALED = database_swissmetro.DefineVariable('CAR_TT_SCALED', CAR_TT / 100)
CAR_CO_SCALED = database_swissmetro.DefineVariable('CAR_CO_SCALED', CAR_CO / 100)



Defining Model Parameters

In [30]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1)
B_COST = Beta('B_COST', 0, None, None, 0)

Defining Random parameter, with Halton Draw, for Monte-Carlo Simulation

In [31]:
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_TIME_S = Beta('B_TIME_S', 1, None, None, 0)
B_TIME_RND = B_TIME + B_TIME_S * bioDraws('B_TIME_RND', 'NORMAL_HALTON2')

Defining the Model

In [32]:
V1 = ASC_TRAIN + B_TIME_RND * TRAIN_TT_SCALED + B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + B_TIME_RND * SM_TT_SCALED + B_COST * SM_COST_SCALED
V3 = ASC_CAR + B_TIME_RND * CAR_TT_SCALED + B_COST * CAR_CO_SCALED

V = {1: V1, 2: V2, 3: V3}

av = {1: TRAIN_AV_SP, 2: SM_AV, 3: CAR_AV_SP}

Estimating the Model

In [34]:
prob = models.logit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))

USER_NOTES = (
    'Example of a mixture of logit models with three alternatives, '
    'approximated using Monte-Carlo integration.'
)

the_biogeme = bio.BIOGEME(
    database_swissmetro, logprob, userNotes=USER_NOTES, parameter_file='few_draws.toml'
)
the_biogeme.modelName = 'b24halton_mixture'

results = the_biogeme.estimate()


In [35]:
print(results.short_summary())

Results for model b24halton_mixture
Nbr of parameters:		5
Sample size:			6768
Excluded data:			3960
Final log likelihood:		-5215.466
Akaike Information Criterion:	10440.93
Bayesian Information Criterion:	10475.03



In [36]:
pandas_results = results.getEstimatedParameters()
pandas_results

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.137538,0.051839,2.653158,7.974250e-03
ASC_TRAIN,-0.401683,0.065680,-6.115730,9.611620e-10
B_COST,-1.284255,0.086268,-14.886829,0.000000e+00
B_TIME,-2.259773,0.117902,-19.166486,0.000000e+00
B_TIME_S,-1.660660,0.132287,-12.553492,0.000000e+00
